# Install packages and imports

In [1]:
!git clone "https://github.com/PredelinaAsya/ModelCompression.git"
%cd ModelCompression/hw5
!pip3 install -qr "requirements.txt"

Cloning into 'ModelCompression'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 64 (delta 27), reused 51 (delta 16), pack-reused 0
Receiving objects: 100% (64/64), 19.69 KiB | 3.94 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/ModelCompression/hw5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.6/615.6 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 16.3 M

In [2]:
from ModelCompression.model_metrics import get_model_size, get_model_sparsity, get_yolo_metrics
import os
import torch
from ultralytics import YOLO
import mlflow

##Mlflow tracking server connection

In [3]:
USER_NAME = "PredelinaAsya" #@param {type:"string"}

#@title Enter the access token:

ACCESS_TOKEN = "f2ebeaf63f365d01d78564af598eb69c8561ce60" #@param {type:"string"}

os.environ['MLFLOW_TRACKING_USERNAME'] = USER_NAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = ACCESS_TOKEN

mlflow.set_tracking_uri('https://dagshub.com/PredelinaAsya/ModelCompression.mlflow')

# ONNX: export and inference

In [4]:
base_model = YOLO('yolov8n.pt')
onnx_path = base_model.export(format="onnx", device='cpu')

100%|██████████| 6.23M/6.23M [00:00<00:00, 53.3MB/s]
Ultralytics YOLOv8.0.190 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 1.4s, saved as 'yolov8n.onnx' (12.2 MB)

Export complete (4.0s)
Results saved to /content/ModelCompression/hw5
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [5]:
def get_file_size_in_mb(path: str):
  size_in_bytes = os.path.getsize(path)
  size_in_mb = size_in_bytes / 2 ** 20
  return size_in_mb

onnx_model = YOLO(onnx_path, task='detect')

with mlflow.start_run(run_name="onnx") as run:
  mlflow.log_param("compression_type", "onnx_convert")
  mlflow.log_metric("model_size", get_file_size_in_mb(onnx_path))
  mlflow.log_param("device", "cpu")
  mlflow.log_param("model_name", "yolov8n")

  onnx_metrics = onnx_model.val(data="coco128.yaml", device='cpu')
  onnx_metrics = get_yolo_metrics(onnx_metrics)
  for key, val in onnx_metrics.items():
    mlflow.log_metric(key, val)


Ultralytics YOLOv8.0.190 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
Loading yolov8n.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models

Dataset 'coco128.yaml' images not found ⚠️, missing path '/content/ModelCompression/hw5/datasets/coco128/images/train2017'
100%|██████████| 6.66M/6.66M [00:00<00:00, 53.9MB/s]
Unzipping /content/ModelCompression/hw5/datasets/coco128.zip to /content/ModelCompression/hw5/datasets/coco128...: 100%|██████████| 263/263 [00:00<00:00, 999.74file/s]
Dataset download success ✅ (1.4s), saved to /content/ModelCompression/hw5/datasets

100%|██████████| 755k/755k [00:00<00:00, 13.4MB/s]
val: Scanning /content/ModelCompression/hw5/datasets/coco128/labels/train2017... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 533.91it/s]
val: New cache created: /content/ModelCompression/hw5/datasets/coco128/labels/train2017.cache
                 Class     Images  Instances      

# OpenVINO: export and inference

In [6]:
openvino_path = base_model.export(format="openvino")

Ultralytics YOLOv8.0.190 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 1.0s, saved as 'yolov8n.onnx' (12.2 MB)


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================




OpenVINO: starting export with openvino 2023.1.0-12185-9e6b00e51cd-releases/2023/1...
OpenVINO: export success ✅ 1.9s, saved as 'yolov8n_openvino_model/' (12.3 MB)

Export complete (4.8s)
Results saved to /content/ModelCompression/hw5
Predict:         yolo predict task=detect model=yolov8n_openvino_model imgsz=640  
Validate:        yolo val task=detect model=yolov8n_openvino_model imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


In [8]:
openvino_model = YOLO(openvino_path, task='detect')

with mlflow.start_run(run_name="openvino") as run:
  mlflow.log_param("compression_type", "openvino_convert")
  mlflow.log_metric("model_size", get_file_size_in_mb(os.path.join(openvino_path, 'yolov8n.bin')))
  mlflow.log_param("device", "cpu")
  mlflow.log_param("model_name", "yolov8n")

  openvino_metrics = openvino_model.val(data="coco128.yaml", device='cpu')
  openvino_metrics = get_yolo_metrics(openvino_metrics)
  for key, val in openvino_metrics.items():
    mlflow.log_metric(key, val)


Ultralytics YOLOv8.0.190 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
Loading yolov8n_openvino_model for OpenVINO inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models
val: Scanning /content/ModelCompression/hw5/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:31<00:00,  4.01it/s]
                   all        128        929      0.651      0.532      0.606      0.452
                person        128        254      0.805      0.667      0.764      0.543
               bicycle        128          6      0.661      0.328      0.329      0.232
                   car        128         46      0.817      0.196      0.269      0.181
            motorcycle        128          5      0.602        0.8       0.88      0.672
              airplane        128   